### Iinstall dependencies by navigating to the IMMLU folder and typing
```pip install -r requirements.txt```

In [ ]:
from datasets import load_dataset
import googletrans
import os
import glob
import time
from datetime import datetime

In [42]:
#https://huggingface.co/datasets/lighteval/mmlu
dataset = load_dataset('lighteval/mmlu', 'all', split = 'auxiliary_train')
df_mmlu = dataset.to_pandas()
df_mmlu.head(5)

,question,subject,choices,answer
0,Davis decided to kill Adams. He set out for Ad...,,"[Adams only., Brooks only., Case only., Adams ...",1
1,A state statute requires any person licensed t...,,"[guilty, because this is a public welfare offe...",3
2,"Lender met Borrower on the street, demanded th...",,"[Yes, because Mann threatened to use deadly fo...",2
3,Peter sued Don for breach of contract. The cou...,,[must permit Don to answer if he had objected ...,1
4,Ames had painted Bell's house under a contract...,,[partial breach of contract only if Ames had p...,2


In [ ]:
os.chdir("../data") #Navigate to the data folder
for i in range(51, 61):
    sample_df = df_mmlu[100*(i-1):100*i]
    translated_df = sample_df.copy()
    translator = googletrans.Translator()
    for index, row in sample_df.iterrows():
        translated_df.loc[index, 'question'] = translator.translate(row['question'], dest='ta').text
        translated_df.loc[index, 'choices'] = translator.translate(row['choices'], dest='ta').text
    translated_df.to_csv(f'translated_mmlu_{(i-1)*100+1}_{i * 100}.csv', index=False)

In [63]:
#Compressing all CSV files
import tarfile
os.chdir("../data") #Navigate to the data folder
t_stamp = f'{datetime.now():%Y-%m-%d %H:%M:%S}'
tar = tarfile.open(f"translated_mmlu_{t_stamp}.tar.gz", "w:gz")
tar.add(os.getcwd(), arcname = f"translated_mmlu_{t_stamp}")
tar.close()

In [68]:
#Delete all CSV files except the compressed file

file_path = f'{os.getcwd()}/translated_mmlu_{t_stamp}.tar.gz'
while not os.path.exists(file_path):
    time.sleep(10)

if os.path.isfile(file_path):
    # Verifies CSV file was created, then deletes unneeded files.
    for clean_up in glob.glob(f'{os.getcwd()}/*.csv'):
        print(clean_up)
        if not clean_up.endswith('tar.gz'):    
            os.remove(clean_up)